In this hands-on we will use the `smolagents` library developed by HuggingFace. We use this library due to its simplicity, support for any LLM posted on HuggingFace Hub and its integration of Code Agents (more on that later).

<img src="https://camo.githubusercontent.com/c6efa99360afde7cf829dff3cad81e56573658c1843464dff1fbb30a8f63b082/68747470733a2f2f68756767696e67666163652e636f2f64617461736574732f68756767696e67666163652f646f63756d656e746174696f6e2d696d616765732f7265736f6c76652f6d61696e2f736d6f6c6167656e74732f736d6f6c6167656e74732e706e67" alt="drawing" width="400"/>

[Smolagents Documentation](https://huggingface.co/docs/smolagents/en/index)

In this hands-on we will:
- Understand why it's helpful to have agentic capabilities
- Understand how to use the `smolagents` library
- Understand the difference between a Tool Calling Agent and a Code Agent
- Implement a custom Agent leveraging the RAG pipeline that we implemented before

## 1. Setting up the environment

We will use `gpt-4o-mini` as our LLM for this hands-on. We could also use any model available on HuggingFace. 

In [ ]:
import sys
sys.path.append("../../")

0

In [ ]:
!pip install -r ../../requirements.txt

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from smolagents import LiteLLMModel
from IPython.display import display_markdown

model = LiteLLMModel(model_id="gpt-4o-mini")

c:\Users\sieverin\Anaconda3\envs\advanced-llm-workshop\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def compute_agent_cost(agent, model_name="gpt-4o-mini"):
    if model_name == "gpt-4o-mini":
        input_token_price = 0.15 / 1000000
        output_token_price = 0.6 / 1000000
    elif model_name == "gpt-4o":
        input_token_price = 2.5 / 1000000
        output_token_price = 10 / 1000000
    return (
        input_token_price * agent.monitor.get_total_token_counts()["input"]
        + output_token_price * agent.monitor.get_total_token_counts()["output"]
    )

## 2. Let's create our first Agent

## 2.1 Tool Calling Agent

In [5]:
from smolagents import ToolCallingAgent

# This is as simple as
agent = ToolCallingAgent(
    tools=[],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)

In [6]:
output = agent.run("What can you do?")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I can assist with a variety of tasks such as answering questions, providing information, generating text, processing data, and more. Please specify a particular task you'd like help with!

Cost of the agent:  0.00015555


Ok, so our agent says that it can help us answering questions. Let's see how it goes.

In [7]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

I'm unable to browse the internet or visit external websites. However, you can check the recent news by visiting https://www.swissinfo.ch/eng/ directly.

Cost of the agent:  0.00015675000000000002


In [8]:
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [9]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

I guess we forgot to give tools to our agent. Let's add a websearch tool. We can use the one provided by default by the smollagents library.

In [10]:
from smolagents import VisitWebpageTool

visit_webpage_tool = VisitWebpageTool()
agent = ToolCallingAgent(
    tools=[visit_webpage_tool],
    model=model,
    verbosity_level=0,
    description="An agent that is capable of searching the web",
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name          ┃ Description                                  ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ visit_webpage │ Visits a webpage at the given url and reads  │ url (`string`): The url of the webpage to    │
    │               │ its content as a markdown string. Use this   │ visit.                                       │
    │               │ to browse webpages.                          │                                              │
    │ final_answer  │ Provides a final answer to the given         │ answer (`any`): The final answer to the      │
    │               │ problem.                                     │ problem                                      │
    └───────────────┴──────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [11]:
output = agent.run(
    "Can you visit https://www.swissinfo.ch/eng/ and tell me what are recent news?"
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

Recent news from Switzerland includes:  
1. **Swiss government backs down to industry outcry over energy security plans** - [Read more](https://www.swissinfo.ch/eng/swiss-politics/federal-council-waives-stricter-rules-for-the-electricity-sector-for-the-time-being/89319692)  
2. **Secure AI system recommended for Swiss parliament** - [Read more](https://www.swissinfo.ch/eng/swiss-politics/a-committee-wants-a-dedicated-ai-assistant-in-the-federal-parliament/89316842)  
3. **Switzerland sets aside more funds to protect religious minorities** - [Read more](https://www.swissinfo.ch/eng/swiss-politics/federal-council-allocates-more-money-for-the-protection-of-religious-minorities/89318974)  
4. **Suspect bombers with Russian links arrested in Switzerland** - [Read more](https://www.swissinfo.ch/eng/international-cooperation/suspect-bombers-with-russian-links-arrested-in-switzerland/89318838)  
5. **Proposed Swiss surveillance law ‘identical to Russia’** - [Read more](https://www.swissinfo.ch/eng/international-geneva/proposed-swiss-surveillance-law-identical-to-russia/89317770)  
6. **Bonuses cuts of ex- Credit Suisse bosses ruled unlawful** - [Read more](https://www.swissinfo.ch/eng/various/bonuses-of-former-credit-suisse-managers-may-not-be-reduced/89316424)  
7. **Swiss pension funds had a good year in 2024** - [Read more](https://www.swissinfo.ch/eng/various/a-good-year-for-swiss-pension-funds-in-2024/89311756)

Cost of the agent:  0.0021765


#### So now we have an agent that can answer questions using the LLM and also search the web for us.
#### Let's see what the agents does behind the scene.

In [12]:
agent.write_memory_to_messages()

[{'role': <MessageRole.SYSTEM: 'system'>,
  'content': [{'type': 'text',
    'text': 'You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to some tools.\n\nThe tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".\nThis Action/Observation can repeat N times, you should take several steps when needed.\n\nYou can use the result of the previous action as input for the next action.\nThe observation will always be a string: it can represent a file, like "image_1.jpg".\nThen you can use it as input for the next action. You can do it for instance as follows:\n\nObservation: "image_1.jpg"\n\nAction:\n{\n  "name": "image_transformer",\n  "arguments": {"image": "image_1.jpg"}\n}\n\nTo provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you

In [13]:
# Let's print the system prompt
print("""""", agent.system_prompt)

 You are an expert assistant who can solve any task using tool calls. You will be given a task to solve as best you can.
To do so, you have been given access to some tools.

The tool call you write is an action: after the tool is executed, you will get the result of the tool call as an "observation".
This Action/Observation can repeat N times, you should take several steps when needed.

You can use the result of the previous action as input for the next action.
The observation will always be a string: it can represent a file, like "image_1.jpg".
Then you can use it as input for the next action. You can do it for instance as follows:

Observation: "image_1.jpg"

Action:
{
  "name": "image_transformer",
  "arguments": {"image": "image_1.jpg"}
}

To provide the final answer to the task, use an action blob with "name": "final_answer" tool. It is the only way to complete the task, else you will be stuck on a loop. So your final output should look like this:
Action:
{
  "name": "final_answer

## Tools

A tool is an atomic function to be used by an agent. To be used by an LLM, it also needs a few attributes that constitute its API and will be used to describe to the LLM how to call this tool:

    A name
    A description
    Input types and descriptions
    An output type

The library provide a list of default tools: https://github.com/huggingface/smolagents/blob/28cfef22389a2830176b48be9fcc3e3d5793b87b/src/smolagents/default_tools.py#L102

- PythonInterpreterTool
- FinalAnswerTool
- UserInputTool
- DuckDuckGoSearchTool
- GoogleSearchTool
- VisitWebpageTool

With the `smolagents` library, there are two ways of declaring tool. Using the `@tool` decorator or using the `Tool` class.

The `@tool` decorator is a more concise way of declaring a tool, but it is less flexible than the `Tool` class.

###  Defining a Tool as a Python Class

In this class, we define:

- `name`: The tool’s name.
- `description`: A description used to populate the agent’s system prompt.
- `inputs`: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
- `output_type`: Specifies the expected output type.
- `forward`: The method containing the inference logic to execute.

In [14]:
from smolagents import Tool


class Sum(Tool):
    name = "sum"
    description = "This is a tool that can add two numbers. It returns the sum of the two numbers."
    inputs = {
        "number_1": {"type": "number", "description": "The first number to add."},
        "number_2": {"type": "number", "description": "The second number to add."},
    }
    output_type = "number"

    def forward(self, number_1: float, number_2: float) -> float:
        return number_1 + number_2

In [15]:
sum_tool = Sum()
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum          │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [16]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum' with arguments: {'number_1': 3, 'number_2': 4}                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 3.03 seconds| Input tokens: 973 | Output tokens: 22]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.06 seconds| Input tokens: 2,044 | Output tokens: 37]

7

Cost of the agent:  0.00032879999999999997


### The @tool Decorator

Using this approach, we define a function with:

- **A clear and descriptive function name** that helps the LLM understand its purpose.
- **Type hints for both inputs and outputs** to ensure proper usage.
- **A detailed description**, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.

In [17]:
from smolagents import tool


@tool
def sum_tool(number_1: float, number_2: float) -> float:
    """
    This is a tool that can add two numbers. It returns the sum of the two numbers.

    Args:
        number_1: The first number to add.
        number_2: The second number to add.
    """
    return number_1 + number_2

In [18]:
agent = ToolCallingAgent(
    tools=[sum_tool],
    model=model,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [19]:
output = agent.run("sum 3 4")
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ sum 3 4                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_tool' with arguments: {'number_1': 3, 'number_2': 4}                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 7

[Step 1: Duration 1.08 seconds| Input tokens: 975 | Output tokens: 23]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '7'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: 7

[Step 2: Duration 1.06 seconds| Input tokens: 2,050 | Output tokens: 38]

7

Cost of the agent:  0.0003303


## Code Agent vs Tool Calling Agent

In [20]:
from datetime import date

today = date.today()
current_date = today.strftime("%Y-%m-%d")

In [21]:
from smolagents import DuckDuckGoSearchTool

web_search = DuckDuckGoSearchTool()
agent = ToolCallingAgent(
    model=model,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

ToolCallingAgent | gpt-4o-mini
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [22]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-14. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'web_search' with arguments: {'query': 'Nvidia stock price on 2025-05-07'}                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Error executing tool 'web_search' with arguments {"query": "Nvidia stock price on 2025-05-07"}: 
DuckDuckGoSearchException: https://lite.duckduckgo.com/lite/ 202 Ratelimit
Please try again or use another tool

[Step 1: Duration 2.11 seconds| Input tokens: 1,110 | Output tokens: 26]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The closing price of Nvidia stock on 2025-05-07 was    │
│ $113.28 and on 2025-05-14 was $136.43. The gain in the last week is $136.43 - $113.28 = $23.15.'}               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: The closing price of Nvidia stock on 2025-05-07 was $113.28 and on 2025-05-14 was $136.43. The gain 
in the last week is $136.43 - $113.28 = $23.15.

[Step 4: Duration 1.78 seconds| Input tokens: 8,862 | Output tokens: 230]

The closing price of Nvidia stock on 2025-05-07 was $113.28 and on 2025-05-14 was $136.43. The gain in the last week is $136.43 - $113.28 = $23.15.

Cost of the agent:  0.0014673


In [23]:
from smolagents import CodeAgent

agent = CodeAgent(
    model=model,
    add_base_tools=False,
    tools=[sum_tool, web_search],
    verbosity_level=1,
    max_steps=10,
)
agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
    ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
    │              │ returns the sum of the two numbers.           │ add.                                         │
    │              │                                               │ number_2 (`number`): The second number to    │
    │              │                                               │ add.                                         │
    │ web_search   │ Performs a duckduckgo web search based on     │ query (`string`): The search query to        │
    │              │ your query (think a Google search) then       │ perform.                                     │
    │              │ returns the top search results.               │                                              │
    │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
    │              │                                               │ problem                                      │
    └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘

In [24]:
output = agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent, model.model_id))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-14. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_price_query = "Nvidia stock price on 2025-05-14"                                                         
  previous_price_query = "Nvidia stock price on 2025-05-07"                                                        
  current_price_info = web_search(current_price_query)                                                             
  print("Current Nvidia stock price information:", current_price_info)                                             
                                                                                                                   
  previous_price_info = web_search(previous_price_query)                                                           
  print("Previous Nvidia stock price information:", previous_price_info)                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Current Nvidia stock price information: ## Search Results

[Nvidia Stock Price Prediction 2025, 2026, 2027-2029](https://longforecast.com/nvidia)
Nvidia stock price prediction. ... Nvidia share outlook for near years. NVDA. NVIDIA STOCK PRICE PREDICTION 2025, 
2026, 2027-2029. Updated: 2025/05/13. Nvidia stock closed at 123.00 dollars ... minimum 163. Averaged Nvidia stock 
price for the month 179. At the end 187 dollars, change for January 14.7%. Nvidia stock prediction for ...

[NVIDIA (NASDAQ: NVDA) Price Prediction and Forecast 2025-2030 for May 
13](https://247wallst.com/forecasts/2025/05/13/nvidia-nvda-price-prediction-and-forecast/)
By the end of 2025, we project shares of NVIDIA to be trading for $137.50, or 5.81% upside potential over the next 
12 months based on a projected EPS of $2.75 and a price-to-earnings (P/E) ratio ...

[NVIDIA STOCK PRICE FORECAST TOMORROW, WEEK, 2025, 2026, 2027 - 30 Rates](https://30rates.com/nvidia)
Nvidia stock price prediction for tomorrow, near days, this week and this month. ... 2025, 2026, 2027. Updated: 
2025/05/13. Nvidia stock closed the previous day at $129.93, which is $19.22 higher than 30 days ago and $16.39 
higher than 7 days ago. Nvidia Stock Price Forecast. ... Nvidia stock price forecast on Wednesday, May, 14: 136.43 
dollars ...

[Nvidia Stock Price In May 2025 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-in-may-2025)
The closing price for NVIDIA this month is $111.61, yesterday. It is down 1.3% for the month. The latest price is 
$114.94. DATE OPEN HIGH LOW CLOSE VOLUME ; May 01 2025. ... nvda stock price march 31st 2025. 2 . dow jones chart 
by jan 2025. 3 . dow jones chart by jan 2025. 4 . dow jones chart by jan 2025. 5 . dow jones chart 2025 year to 
date ...

[NVIDIA (NVDA) Stock Forecast and Price Target 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/forecast/)
Closing price 05/12/2025 04:00 PM Eastern. Extended Trading. $122.83-.17 ... NVIDIA - Analysts' Recommendations and
Stock Price Forecast (2025) ... Here are some ways that investors could benefit from investing in NVidia: The 
current stock price is around $116.83, reflecting strong market interest and potential for growth. ...

[NVIDIA Corporation (NVDA) Stock Historical Prices & Data - Yahoo 
Finance](https://finance.yahoo.com/quote/NVDA/history/)
Discover historical prices for NVDA stock on Yahoo Finance. View daily, weekly or monthly format back to when 
NVIDIA Corporation stock was issued. News ... Jan 14, 2025: 136.05: 136.38: 130.05:

[Nvidia Stock Price History 2025 | StatMuse 
Money](https://www.statmuse.com/money/ask/nvidia-stock-price-history-2025)
The closing price for NVIDIA (NVDA) this year is $117.06, yesterday. It is down 13.9% for the year. The latest 
price is $117.35.

[NVIDIA (NVDA) Historical Stock Price Data - Stock Analysis](https://stockanalysis.com/stocks/nvda/history/)
NVDA News. 7 hours ago - Nvidia, AMD help the chip sector to its highest close since February - Market Watch 8 
hours ago - Nvidia: The Bull Case Just Got Stronger - Seeking Alpha 11 hours ago - China Trade Progress: Market 
Booms On News, Who Will Benefit? - Seeking Alpha ; 14 hours ago - Nasdaq 100: Taiwan Semiconductor, Nvidia, Micron 
Propel Market Surge - FXEmpire

[NVIDIA (NVDA) Stock Chart and Price History 2025 - 
MarketBeat](https://www.marketbeat.com/stocks/NASDAQ/NVDA/chart/)
The NVIDIA (NVDA) stock chart highlights key performance trends across multiple timeframes. Over the last 12 
months, the stock's price has increased 36.06%, with a year-to-date return of -8.41%.In the past month, the stock 
has increased 10.88%, reflecting recent market activity. As of the latest close, NVIDIA traded at $123.00 with a 
market cap of $3.01 trillion and volume of 224.26 million ...

[Nvidia (NVDA) Stock Forecast & Price Predictions for 2025, 2026, 2027 
...](https://www.litefinance.org/blog/analysts-opinions/nvda-stock-price-prediction/)
The Nvidia stock pric

[Step 1: Duration 6.46 seconds| Input tokens: 2,082 | Output tokens: 161]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  current_price = 129.93                                                                                           
  previous_price = 111.61                                                                                          
  gain = current_price - previous_price                                                                            
  print(f"The gain in Nvidia stock over the last week is: {gain}")                                                 
  final_answer(gain)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The gain in Nvidia stock over the last week is: 18.320000000000007

Out - Final answer: 18.320000000000007

[Step 2: Duration 2.61 seconds| Input tokens: 6,880 | Output tokens: 289]

18.320000000000007

Cost of the agent:  0.0012054


## Agent Hierarchy / MultiAgent

It is also possible to use a multi-agent system, where multiple agents can be used to solve a problem. This is useful when the problem is too complex for a single agent to solve. In this case, the agents can communicate with each other to solve the problem.

Another advantage is context size, as a single agent will store the full history of the steps, while a multi-agent system will store only the history of the steps of the agent that is currently active.

In [25]:
from smolagents import CodeAgent, DuckDuckGoSearchTool


web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model,
    add_base_tools=False,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=1,
)

manager_agent = CodeAgent(
    tools=[sum_tool],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=1,
    description="An agent that manages other agent.",
    max_steps=10,
)

manager_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ sum_tool     │ This is a tool that can add two numbers. It   │ number_1 (`number`): The first number to     │
│   │              │ returns the sum of the two numbers.           │ add.                                         │
│   │              │                                               │ number_2 (`number`): The second number to    │
│   │              │                                               │ add.                                         │
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── information_retriever_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: An agent that can be called to run web search to obtain information. Call it as a 
        │   function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name         ┃ Description                               ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search   │ Performs a duckduckgo web search based on │ query (`string`): The search query to    │
            │              │ your query (think a Google search) then   │ perform.                                 │
            │              │ returns the top search results.           │                                          │
            │ final_answer │ Provides a final answer to the given      │ answer (`any`): The final answer to the  │
            │              │ problem.                                  │ problem                                  │
            └──────────────┴───────────────────────────────────────────┴──────────────────────────────────────────┘

In [26]:
output = manager_agent.run(
    f"You are an agent that can study financial market. Today's date is {current_date}. "
    + "What is the gain that Nvidia stock made in the last week?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(manager_agent))

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an agent that can study financial market. Today's date is 2025-05-14. What is the gain that Nvidia      │
│ stock made in the last week?                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_current_price = information_retriever_agent("Current Nvidia stock price on 2025-05-14.")                  
  print("Nvidia current price:", nvidia_current_price)                                                             
                                                                                                                   
  nvidia_last_week_price = information_retriever_agent("Nvidia stock price on 2025-05-07.")                        
  print("Nvidia price a week ago:", nvidia_last_week_price)                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭───────────────────────────────────── New run - information_retriever_agent ─────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'information_retriever_agent'.                                                     │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Current Nvidia stock price on 2025-05-14.                                                                       │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(percentage_gain)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 21.812500000000007

[Step 3: Duration 1.60 seconds| Input tokens: 9,532 | Output tokens: 712]

21.812500000000007

Cost of the agent:  0.001857


## Visualize Your Agent

In [29]:
from smolagents import (
    GradioUI,
)

web_agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="information_retriever_agent",
    description="An agent that can be called to run web search to obtain information. Call it as a function using the **task** argument.",
    verbosity_level=0,
)

manager_agent = CodeAgent(
    name="ManagerAgent",
    tools=[],
    model=model,
    managed_agents=[web_agent],
    verbosity_level=0,
    description="An agent that manages other agent.",
    max_steps=10,
)

GradioUI(manager_agent).launch(
    share=True
)  # This is necessary to make it work in Renku but be careful

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://7d696e0df8cccd2b9b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7d696e0df8cccd2b9b.gradio.live


# !!! Do not forget to stop the process of the previous cell before executing the next ones !!! 

## Another Multi Agent Example

We will create:

- an agent that can read a document and summarize it

- an agent that can search the web

- a manager agent that handle both previous agents

In [30]:
import requests
from helpers.data_processing import SimpleChunker, PDFExtractorAPI


@tool
def read_pdf_tool(pdf_file_path: str) -> str:
    """
    This tool reads a PDF file and returns the text content of the PDF file.

    Args:
        pdf_file_path: The path to the PDF file.
    """
    response, text, images = PDFExtractorAPI().extract_text_and_images(pdf_file_path)
    return text

In [31]:
tmp = read_pdf_tool("../../data/hypoxy_stat_1page.pdf")
tmp

'Please cite this article in press as: Blume et al., HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity, Cell (2025), https://doi.org/10.1016/j.cell.2025.01.029\n\n![](_page_0_Picture_2.jpeg)\n\n### Article HypoxyStat, a small-molecule form of hypoxia therapy that increases oxygen-hemoglobin affinity\n\nSkyler Y. Blume,[1,](#page-0-0)[2](#page-0-1) Ankur Garg,[1,](#page-0-0)[2](#page-0-1) Yolanda Martı´-Mateos,[1,](#page-0-0)[2](#page-0-1) Ayush D. Midha,[1,](#page-0-0)[2](#page-0-1) Brandon T. L. Chew,[1,](#page-0-0)[2](#page-0-1) Baiwei Lin,[3](#page-0-2) Cecile Yu,[3](#page-0-2) Ryan Dick,[3](#page-0-2) Patrick S. Lee,[3](#page-0-2) Eva Situ,[3](#page-0-2) Richa Sarwaikar,[3](#page-0-2) Eric Green,[3](#page-0-2) Vyas Ramanan,[3](#page-0-2) Gijsbert Grotenbreg,[3](#page-0-2) Maarten Hoek,[3](#page-0-2) Christopher Sinz,[3](#page-0-2) and Isha H. Jain[1,](#page-0-0)[2,](#page-0-1)[4,](#page-0-3)[5,](#page-0-4)[\\\\*](#page-0-5)\n\n<span id="

In [32]:
agent_read_pdf = CodeAgent(
    name="read_pdf_agent",
    description="Reads and summarize a PDF file. Call it as a function using the **task** argument.",
    tools=[read_pdf_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent_web_search = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

agent = CodeAgent(
    name="medical_agent",
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[agent_read_pdf, agent_web_search],
    verbosity_level=1,
)

agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── read_pdf_agent | CodeAgent | gpt-4o-mini
    │   ├── ✅ Authorized imports: []
    │   ├── 📝 Description: Reads and summarize a PDF file. Call it as a function using the **task** argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ read_pdf_tool │ This tool reads a PDF file and returns   │ pdf_file_path (`string`): The path to    │
    │       │               │ the text content of the PDF file.        │ the PDF file.                            │
    │       │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │       │               │ on your query (think a Google search)    │ perform.                                 │
    │       │               │ then returns the top search results.     │                                          │
    │       │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │       │               │ reads its content as a markdown string.  │ to visit.                                │
    │       │               │ Use this to browse webpages.             │                                          │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── web_search_agent | CodeAgent | gpt-4o-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you. Call it as a function using the **task** argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description                              ┃ Arguments                                ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
            │               │ on your query (think a Google search)    │ perform.                                 │
            │               │ then returns the top search results.     │                                          │
            │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
            │               │ reads its content as a markdown string.  │ to visit.                                │
            │               │ Use this to browse webpages.             │                                          │
            │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  

In [33]:
output = agent.run(
    "Your task is the following: "
    + "Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it is about? Also, can you give me the wikipedia definition of the area of research?",
    reset=True,
)
display_markdown(output, raw=True)
print("Cost of the agent: ", compute_agent_cost(agent))

╭──────────────────────────────────────────── New run - medical_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ Your task is the following: Can you read the PDF file at '../../data/hypoxy_stat_1page.pdf' and tell me what it │
│ is about? Also, can you give me the wikipedia definition of the area of research?                               │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pdf_summary = read_pdf_agent("Please read and summarize the content of the PDF file at                           
  '../../data/hypoxy_stat_1page.pdf'.")                                                                            
  print("Summary of the PDF:", pdf_summary)                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────────── New run - read_pdf_agent ────────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'read_pdf_agent'.                                                                  │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Please read and summarize the content of the PDF file at '../../data/hypoxy_stat_1page.pdf'.                    │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_summary = (                                                                                                
      "Summary of the PDF: HypoxyStat is a small-molecule therapy that enhances oxygen transportation and shows    
  promise for mitochondrial diseases like Leigh syndrome. It improves lifespan and health in mouse models by       
  inducing systemic hypoxia under normal oxygen conditions.\n"                                                     
      "Wikipedia definition of mitochondrial diseases: Mitochondrial diseases are a group of disorders caused by   
  mitochondrial dysfunction affecting energy production in cells. They can cause various symptoms including        
  muscle weakness and neurological problems, and currently, there is no cure, though treatments focus on symptom   
  management."                                                                                                     
  )                                                                                                                
  final_answer(final_summary)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Summary of the PDF: HypoxyStat is a small-molecule therapy that enhances oxygen transportation 
and shows promise for mitochondrial diseases like Leigh syndrome. It improves lifespan and health in mouse models 
by inducing systemic hypoxia under normal oxygen conditions.
Wikipedia definition of mitochondrial diseases: Mitochondrial diseases are a group of disorders caused by 
mitochondrial dysfunction affecting energy production in cells. They can cause various symptoms including muscle 
weakness and neurological problems, and currently, there is no cure, though treatments focus on symptom management.

[Step 3: Duration 4.17 seconds| Input tokens: 11,195 | Output tokens: 1,051]

Summary of the PDF: HypoxyStat is a small-molecule therapy that enhances oxygen transportation and shows promise for mitochondrial diseases like Leigh syndrome. It improves lifespan and health in mouse models by inducing systemic hypoxia under normal oxygen conditions.
Wikipedia definition of mitochondrial diseases: Mitochondrial diseases are a group of disorders caused by mitochondrial dysfunction affecting energy production in cells. They can cause various symptoms including muscle weakness and neurological problems, and currently, there is no cure, though treatments focus on symptom management.

Cost of the agent:  0.00230985


## Exercise 1: Create your own RAG Agent

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/blog/beating_gaia/classical_vs_agentic_rag.png)

First we will create a simple agent that can answer questions on a knowledge base, AKA a **RAG agent**.

1. Define a tool that retrieve documents from a knowledge base
2. Define an agent that uses the tool to retrieve documents and answer questions

In [34]:
from helpers.constants_and_data_classes import Chunk
from helpers.data_processing import SimpleChunker, PDFExtractorAPI
from helpers.embedding import (
    OpenAITextEmbeddings,
    compute_openai_large_embedding_cost,
)
from helpers.vectorstore import (
    ChromaDBVectorStore,
    VectorStoreRetriever,
)

### Retriever Pipeline

In [36]:
data_folder = "../../data"

pdf_files = [
    "Explainable_machine_learning_prediction_of_edema_a.pdf",
    "Modeling tumor size dynamics based on real‐world electronic health records.pdf",
]
example_pdf_file = "Explainable_machine_learning_prediction_of_edema_a.pdf"
example_pdf_path = os.path.join(data_folder, example_pdf_file)

vector_store_collection = "text_collection"

In [37]:
data_extractor = PDFExtractorAPI()
_, text, _ = data_extractor.extract_text_and_images(example_pdf_path)

In [38]:
file_metadata = {"source_text": example_pdf_file}

text_chunker = SimpleChunker(max_chunk_size=1000)

chunks = text_chunker.chunk_text(text, file_metadata)

In [39]:
embedding_model = OpenAITextEmbeddings()
embeddings = embedding_model.get_embedding([chunk.content for chunk in chunks])

In [40]:
vector_store = ChromaDBVectorStore(vector_store_collection)
vector_store.insert_chunks(chunks, embeddings)

In [41]:
retriever = VectorStoreRetriever(embedding_model, vector_store)
results = retriever.retrieve("Who are the authors of the paper=", 5)
results

[[{'chunk_id': '48',
   'score': 1.2668571472167969,
   'chunk': Chunk(chunk_id=48, content='performed the research. F.A. and N.T. analyzed the data. F.A., N.T., and R.C. contributed new reagents/analytical tools. ### **FUNDING INFORMATION** This study was sponsored by the healthcare business of Merck KGaA, Darmstadt, Germany (CrossRef Funder ID:10.13039/100009945). Editorial support was provided by Syneos Health, London, UK, and funded by the healthcare business of Merck KGaA, Darmstadt, Germany. ### **CONFLICT OF INTEREST STATEMENT** N.T. is an employee of Ares Trading S.A., Lausanne, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. R.S., R.B. and A.J. are employees of the healthcare business of Merck KGaA, Darmstadt, Germany. K.B. was an employee of the healthcare business of Merck KGaA, Darmstadt, Germany at the time of the study. V.G. is an employee of Ares Trading S.A. Eysins, Switzerland, an affiliate of Merck KGaA, Darmstadt, Germany. F.A. and R.C.', metadata={'sour

### Toolify

In [43]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.

    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It returns a list of string │ chunks to return, by default it's 5.        │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [44]:
# Create the retriever tool
@tool
def retriever_tool(query: str, number_of_chunks: int) -> list:
    """
    This is a tool that can search a document and extract the related information based on the query given. It returns a list of string
    Only use this tool if necessary to answer the query otherwise rely on your internal knowledge.

    Args:
        query: The user query
        number_of_chunks: number of chunks to return, by default it's 5.
    """

    return retriever.retrieve(query, number_of_chunks)


# Create the CodeAgent with the tool
rag_agent = CodeAgent(tools=[retriever_tool], model=model)
rag_agent.visualize()

CodeAgent | gpt-4o-mini
├── ✅ Authorized imports: []
└── 🛠️ Tools:
    ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    ┃ Name           ┃ Description                                  ┃ Arguments                                   ┃
    ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │ retriever_tool │ This is a tool that can search a document    │ query (`string`): The user query            │
    │                │ and extract the related information based on │ number_of_chunks (`integer`): number of     │
    │                │ the query given. It returns a list of string │ chunks to return, by default it's 5.        │
    │                │ Only use this tool if necessary to answer    │                                             │
    │                │ the query otherwise rely on your internal    │                                             │
    │                │ knowledge.                                   │                                             │
    │ final_answer   │ Provides a final answer to the given         │ answer (`any`): The final answer to the     │
    │                │ problem.                                     │ problem                                     │
    └────────────────┴──────────────────────────────────────────────┴─────────────────────────────────────────────┘

In [45]:
# Call the agent
output = rag_agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  shap_analysis_factors = retriever_tool(query="SHAP analysis higher-grade edema Grade 2+ factors",                
  number_of_chunks=5)                                                                                              
  print(shap_analysis_factors)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[[{'chunk_id': '39', 'score': 0.6222168207168579, 'chunk': Chunk(chunk_id=39, content='Points are colored based on 
the edema grade at the following safety visit. SHAP, Shapley Additive exPlanations. with higher grades of edema, 
particularly grade 2+. On the other hand, for higher albumin levels the corresponding SHAP values are mostly 
negative and ranging from 0 to −0.5, suggesting a reduced risk of edema of grade 2+. The association between age 
greater than 70years and an increased likelihood of edemas of grades 2+ was also confirmed. Additionally, for all 
ages, higher SHAP values were assigned to patients who experienced edemas, particularly of grade 2+. Finally, 
within low ranges of cumulated dose in the interval [ *t* − 14 days, *t* ] normalized over 14days, higher SHAP 
values were assigned to samples corresponding to edemas of grades 2+. This could reflect the tendency to adjust 
administered doses in those cases where the risk of edema was identified. # **DISCUSSION**', 
metadata={'source_text': 'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 39}, 
data_type=<DataType.TEXT: 'text'>, score=None)}, {'chunk_id': '37', 'score': 0.8699163794517517, 'chunk': 
Chunk(chunk_id=37, content='![](_page_7_Figure_0.jpeg) <span id="page-7-0"></span>**FIGURE 2** Global input 
importance via mean SHAP values. Ranking of the model input for the most influential to the less influential for 
the model. The y-axis indicates the average change in the predicted probability of edema by grade, on average 
across the entire test set. SHAP, Shapley Additive exPlanations. <span id="page-7-1"></span>**FIGURE 3** SHAP 
values – contribution of the inputs toward the predicted probabilities of edemas of grade 2+. List of the eight 
most influential inputs with respect to the predicted probabilities of edemas of grades 2+. Each point on the plot 
is a SHAP value for a covariate at a specific patient visit. The position on the y-axis indicates the covariate 
importance and on the x-axis the impact on the predicted probability. Color represents the value of the covariate. 
SHAP, Shapley Additive exPlanations. with an increase in the predicted probability of edemas of grades 2+.', 
metadata={'source_text': 'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 37}, 
data_type=<DataType.TEXT: 'text'>, score=None)}, {'chunk_id': '42', 'score': 0.883253812789917, 'chunk': 
Chunk(chunk_id=42, content='The presence of such input is ensuring high model performances when predicting 
persistence of a given edema grade. The sensitivity analysis to the inclusion of this input revealed a decline in 
the mean cross-validation F1 score of ~0.350 when it was removed as a candidate predictor. However, the use of 
Isotonic Regressions ensures the estimations of correctly calibrated probabilities. For instance, while low 
predicted probabilities, for example, for edemas of grade 2+, might lead to a classification error, they can still 
provide valuable details on existing risk of occurrence of an adverse event (Figure [S5\\)](#page-11-0). Moreover, 
sensitivity analysis together with the SHAP importance highlighted interesting patterns with respect to the 
exposure-related features. Referencing Figure [3](#page-7-1), lower values of the dose [ *t* − 14 days, *t* ] are 
associated with a decreased probability of edema of grades 2+ at subsequent visits.', metadata={'source_text': 
'Explainable_machine_learning_prediction_of_edema_a.pdf', 'document_chunk_id': 42}, data_type=<DataType.TEXT: 
'text'>, score=None)}, {'chunk_id': '32', 'score': 0.9377022981643677, 'chunk': Chunk(chunk_id=32, 
content='Consistently with the above sensitivity analysis, past current edema grade was found to be the most 
influential input, particularly if a same grade persisted to the following safety visit. The exposure-derived 
features were also informative for the model probability predictions. Albumin was found as the 

[Step 1: Duration 4.82 seconds| Input tokens: 2,046 | Output tokens: 131]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({                                                                                                   
      "Most Influential Factors for Grade 2+ Edema": [                                                             
          "Current Edema Grade",                                                                                   
          "Albumin Levels",                                                                                        
          "Age greater than 70 years",                                                                             
          "Recent Dose Adjustments"                                                                                
      ]                                                                                                            
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Most Influential Factors for Grade 2+ Edema': ['Current Edema Grade', 'Albumin Levels', 'Age 
greater than 70 years', 'Recent Dose Adjustments']}

[Step 2: Duration 5.49 seconds| Input tokens: 5,699 | Output tokens: 350]

In [46]:
output = rag_agent.run("What is the highest court of the USA?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the highest court of the USA?                                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Supreme Court")                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Supreme Court

[Step 1: Duration 2.32 seconds| Input tokens: 2,032 | Output tokens: 66]

## Exercise 2: Make it a multi-agent system

Now that we built a RAG agent, we will improve it by adding a web search tool to it. This way, if the agent can't find the answer in the knowledge base, it will search the web for it.

However, we will transform our agent into a multi-agent system. This way, we will have one agent responsible for answering questions using the knowledge base and another agent responsible for searching the web.

1. Define a tool that searches the web
2. Define a new agent that uses the web search tool
3. Create a multi-agent system that uses both agents


In [47]:
#
from smolagents import DuckDuckGoSearchTool

web_search_agent = CodeAgent(
    name="web_search_agent",
    description="Runs web searches for you. Call it as a function using the **task** argument.",
    tools=[DuckDuckGoSearchTool(), visit_webpage_tool],
    add_base_tools=True,
    model=model,
    verbosity_level=1,
)

rag_agent = CodeAgent(
    name="medical_literature_agent",
    description="Retrieve information from medical litterature. Call it as a function using the **task** argument.",
    tools=[retriever_tool],
    model=model,
)

agent = CodeAgent(
    tools=[],
    model=model,
    add_base_tools=False,
    managed_agents=[rag_agent, web_search_agent],
    verbosity_level=1,
)

In [48]:
# Call the agent
output = agent.run(
    "According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade 2+)?"
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  factors_influential = medical_literature_agent("According to SHAP analysis, which factors were the most          
  influential in predicting higher-grade edema (Grade 2+)?")                                                       
  print(factors_influential)                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────── New run - medical_literature_agent ───────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'medical_literature_agent'.                                                        │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ According to SHAP analysis, which factors were the most influential in predicting higher-grade edema (Grade     │
│ 2+)?                                                                                                            │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 19.70 seconds| Input tokens: 3,739 | Output tokens: 494]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The most influential factors predicting higher-grade edema (Grade 2+) include: 1. Previous Edema   
  Grade 2. Low Albumin Levels 3. Age above 70 years 4. Recent Cumulated Medication Doses.")                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The most influential factors predicting higher-grade edema (Grade 2+) include: 1. Previous 
Edema Grade 2. Low Albumin Levels 3. Age above 70 years 4. Recent Cumulated Medication Doses.

[Step 2: Duration 2.85 seconds| Input tokens: 6,324 | Output tokens: 626]

In [49]:
output = agent.run("What is the current price of Nvidia stock?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the current price of Nvidia stock?                                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  nvidia_stock_price = web_search_agent("current price of Nvidia stock")                                           
  print(nvidia_stock_price)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭────────────────────────────────────────── New run - web_search_agent ───────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'web_search_agent'.                                                                │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ current price of Nvidia stock                                                                                   │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o-mini ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 1: Duration 16.91 seconds| Input tokens: 7,642 | Output tokens: 335]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(132.49)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 132.49

[Step 2: Duration 1.42 seconds| Input tokens: 10,082 | Output tokens: 379]